Vistra runs multiple power plants, and each plant has an identifer. 
An outage can occur at one or more of our plants and we would want to aggregate our outage time across all our plants.
Outage data is tracked as a list of tuples:
```
(plant_id, outage_start_time, duration(mins))
```
#### Build a function to report total downtime (minutes) in given period of time (window_start, window_end) for any plant.

In [7]:
from datetime import datetime, timedelta

import pandas as pd
 
outages = pd.DataFrame([

    (1, '2023-01-01 00:00:01', 23),

    (1, '2023-01-09 00:00:05',100), 

    (1, '2023-01-21 00:00:01', 25),

    (2, '2023-01-01 23:54:00', 10), #4

    (2, '2023-01-04 00:04:01', 17), #17

    (2, '2023-01-08 23:54:00', 17), #6

    (2, '2023-01-15 00:00:01', 38),

    (2, '2023-01-30 00:00:01',  7),

    (3, '2023-01-08 23:55:00',  7), #5

    (4, '2023-01-08 23:58:00',  5), #2   

    (4, '2023-01-05 08:55:00',  10) #10

])

outages.columns = ['plant_id', 'outage_start_time', 'duration']
 
start_window = datetime(2023, 1, 2) #

end_window = datetime(2023, 1, 9) # exclude

# Solution- plant_id: downtime = (1:0); (2:27); (3:5); (4:12)

In [8]:
outages

,plant_id,outage_start_time,duration
0,1,2023-01-01 00:00:01,23
1,1,2023-01-09 00:00:05,100
2,1,2023-01-21 00:00:01,25
3,2,2023-01-01 23:54:00,10
4,2,2023-01-04 00:04:01,17
5,2,2023-01-08 23:54:00,17
6,2,2023-01-15 00:00:01,38
7,2,2023-01-30 00:00:01,7
8,3,2023-01-08 23:55:00,7
9,4,2023-01-08 23:58:00,5


In [12]:
outages['outage_start_time'] = pd.to_datetime(outages['outage_start_time'])


In [45]:
# outages['outage_end_time'] = outages['outage_start_time'] + p 
# outages['duration']

outages['outage_end_time'] = 0
for i in range(len(outages)):
        outages.iloc[i,3] = outages.iloc[i,1]+pd.Timedelta(minutes=outages.iloc[i,2])
        

outages['outage_end_time'] = pd.to_datetime(outages['outage_end_time'])


C:\Users\gkris\AppData\Local\Temp\ipykernel_42900\196696979.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-01-01 00:23:01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  outages.iloc[i,3] = outages.iloc[i,1]+pd.Timedelta(minutes=outages.iloc[i,2])


In [72]:
# outages[(outages.outage_start_time<end_window) & (outages.outage_end_time>end_window)]
#  outages[(outages.outage_start_time < start_window) & (outages.outage_end_time>start_window)]

condition1 = (outages.outage_start_time<end_window) & (outages.outage_end_time>end_window)
condition2 = (outages.outage_start_time < start_window) & (outages.outage_end_time>start_window)

condition3 = (outages.outage_start_time >= start_window) & (outages.outage_start_time<end_window) & (outages.duration_overlap==0)
outages['duration_overlap'] = 0 

outages.loc[condition2, 'duration_overlap'] = (outages.loc[condition2, 'outage_end_time'] - start_window)
outages.loc[condition1, 'duration_overlap'] =   (end_window - outages.loc[condition1, 'outage_start_time'])

outages['duration_timedelta'] = pd.to_timedelta(outages['duration'], unit='m')

outages['shutdown_duration'] = outages['duration_overlap']

outages.loc[condition3, 'shutdown_duration'] = outages.loc[condition3, 'duration_timedelta']


outages['shutdown_duration'] =  pd.to_timedelta(outages['shutdown_duration'], unit='m')



outages.groupby('plant_id')['shutdown_duration'].sum()

C:\Users\gkris\AppData\Local\Temp\ipykernel_42900\1489625361.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<TimedeltaArray>
['0 days 00:04:00']
Length: 1, dtype: timedelta64[ns]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  outages.loc[condition2, 'duration_overlap'] = (outages.loc[condition2, 'outage_end_time'] - start_window)


plant_id
1   0 days 00:00:00
2   0 days 00:27:00
3   0 days 00:05:00
4   0 days 00:12:00
Name: shutdown_duration, dtype: timedelta64[ns]

In [73]:
def shutdown_plant_duration_summary(df:pd.DataFrame, start_window, end_window):

    ''''Print Summary of plant shutdown duration'''

    condition1 = (outages.outage_start_time<end_window) & (outages.outage_end_time>end_window)
    condition2 = (outages.outage_start_time < start_window) & (outages.outage_end_time>start_window)

    condition3 = (outages.outage_start_time >= start_window) & (outages.outage_start_time<end_window) & (outages.duration_overlap==0)
    outages['duration_overlap'] = 0 

    outages.loc[condition2, 'duration_overlap'] = (outages.loc[condition2, 'outage_end_time'] - start_window)
    outages.loc[condition1, 'duration_overlap'] =   (end_window - outages.loc[condition1, 'outage_start_time'])

    outages['duration_timedelta'] = pd.to_timedelta(outages['duration'], unit='m')

    outages['shutdown_duration'] = outages['duration_overlap']

    outages.loc[condition3, 'shutdown_duration'] = outages.loc[condition3, 'duration_timedelta']


    outages['shutdown_duration'] =  pd.to_timedelta(outages['shutdown_duration'], unit='m')



    display(outages.groupby('plant_id')['shutdown_duration'].sum())


shutdown_plant_duration_summary(outages, start_window, end_window)

C:\Users\gkris\AppData\Local\Temp\ipykernel_42900\2991823388.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<TimedeltaArray>
['0 days 00:04:00']
Length: 1, dtype: timedelta64[ns]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  outages.loc[condition2, 'duration_overlap'] = (outages.loc[condition2, 'outage_end_time'] - start_window)


plant_id
1   0 days 00:00:00
2   0 days 00:27:00
3   0 days 00:05:00
4   0 days 00:12:00
Name: shutdown_duration, dtype: timedelta64[ns]

In [10]:
help(timedelta)

Help on class timedelta in module datetime:

class timedelta(builtins.object)
 |  Difference between two datetime values.
 |  
 |  timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
 |  
 |  All arguments are optional and default to 0.
 |  Arguments may be integers or floats, and may be positive or negative.
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __divmod__(self, value, /)
 |      Return divmod(self, value).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __floordiv__(self, value, /)
 |      Return self//value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  